# Dynamic Policyholder Behavior: Lapse Rate Modeling

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brandonmbehring-dev/insurance-ai-toolkit/blob/main/notebooks/02_behavior_lapse.ipynb)

This notebook explores **dynamic lapse rate modeling** for Variable Annuities with GLWB riders, demonstrating how policyholder behavior varies with market conditions.

## Learning Objectives

1. Understand the concept of "moneyness" for VA guarantees
2. Implement dynamic lapse rate models
3. Visualize rational policyholder behavior
4. Quantify the reserve impact of behavior assumptions

---

## 1. Background: Why Behavior Matters

### The Policyholder's Decision

A VA+GLWB policyholder faces a key decision each year:
- **Stay**: Keep the policy, receive guaranteed withdrawals
- **Lapse**: Surrender the policy, receive account value

### Rational Behavior

Policyholders are economically rational (on average):

| Account Value vs Benefit | Rational Action | Why |
|--------------------------|-----------------|-----|
| AV >> Benefit Base (ITM) | **Stay** | Guarantee is valuable, account is growing |
| AV << Benefit Base (OTM) | **Lapse** | Better to take cash than wait for underwater guarantee |
| AV ≈ Benefit Base (ATM) | **Mixed** | Depends on age, health, alternatives |

### Moneyness

**Moneyness** = Account Value / Benefit Base

| Moneyness | Interpretation |
|-----------|----------------|
| > 1.0 | In-The-Money (ITM) - policyholder winning |
| = 1.0 | At-The-Money (ATM) - break-even |
| < 1.0 | Out-The-Money (OTM) - policyholder underwater |

## 2. Setup

In [ ]:
# Install dependencies (for Colab)
# !pip install numpy pandas matplotlib seaborn --quiet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, List, Callable

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
np.random.seed(42)

print("Setup complete!")

## 3. Dynamic Lapse Models

### Model 1: Static Lapse Rate (Naive)

The simplest assumption: constant lapse rate regardless of market conditions.

In [ ]:
def static_lapse_rate(moneyness: float, base_rate: float = 0.08) -> float:
    """
    Static lapse model - ignores moneyness.
    
    Args:
        moneyness: Account Value / Benefit Base
        base_rate: Annual lapse rate (default 8%)
    
    Returns:
        Annual lapse probability
    """
    return base_rate

# Test
print(f"Static lapse (any moneyness): {static_lapse_rate(0.8):.1%}")

### Model 2: Linear Dynamic Lapse

Lapse rate increases linearly as policy goes out-of-the-money.

In [ ]:
def linear_dynamic_lapse(
    moneyness: float,
    base_rate: float = 0.08,
    sensitivity: float = 0.15,
    floor: float = 0.02,
    cap: float = 0.25,
) -> float:
    """
    Linear dynamic lapse model.
    
    Lapse = base_rate + sensitivity × (1 - moneyness)
    
    Args:
        moneyness: Account Value / Benefit Base
        base_rate: Lapse rate when ATM (moneyness = 1.0)
        sensitivity: How much lapse increases per 1.0 decrease in moneyness
        floor: Minimum lapse rate
        cap: Maximum lapse rate
    
    Returns:
        Annual lapse probability
    """
    lapse = base_rate + sensitivity * (1.0 - moneyness)
    return np.clip(lapse, floor, cap)

# Test at different moneyness levels
test_moneyness = [0.7, 0.85, 1.0, 1.15, 1.3]
for m in test_moneyness:
    lapse = linear_dynamic_lapse(m)
    print(f"Moneyness {m:.2f}: Lapse = {lapse:.1%}")

### Model 3: S-Curve (Logistic) Dynamic Lapse

More realistic: lapse rate follows an S-curve, capturing:
- Low lapse when deep ITM
- Rapid increase as moneyness drops below 1.0
- Saturation at high lapse rates for deep OTM

In [ ]:
def scurve_dynamic_lapse(
    moneyness: float,
    base_rate: float = 0.08,
    max_rate: float = 0.25,
    min_rate: float = 0.02,
    steepness: float = 8.0,
    inflection: float = 0.85,
) -> float:
    """
    S-curve (logistic) dynamic lapse model.
    
    Uses sigmoid function centered at inflection point.
    
    Args:
        moneyness: Account Value / Benefit Base
        base_rate: Lapse rate at inflection point
        max_rate: Maximum lapse rate (deep OTM)
        min_rate: Minimum lapse rate (deep ITM)
        steepness: How sharp the transition is
        inflection: Moneyness at which transition occurs
    
    Returns:
        Annual lapse probability
    """
    # Sigmoid function: higher when moneyness is lower
    z = steepness * (inflection - moneyness)
    sigmoid = 1 / (1 + np.exp(-z))
    
    # Map sigmoid [0,1] to [min_rate, max_rate]
    lapse = min_rate + (max_rate - min_rate) * sigmoid
    return lapse

# Test
for m in test_moneyness:
    lapse = scurve_dynamic_lapse(m)
    print(f"Moneyness {m:.2f}: Lapse = {lapse:.1%}")

## 4. Visualizing Lapse Curves

Let's compare all three models across the moneyness spectrum.

In [ ]:
# Generate moneyness range
moneyness_range = np.linspace(0.5, 1.5, 100)

# Calculate lapse rates for each model
static_lapses = [static_lapse_rate(m) for m in moneyness_range]
linear_lapses = [linear_dynamic_lapse(m) for m in moneyness_range]
scurve_lapses = [scurve_dynamic_lapse(m) for m in moneyness_range]

# Plot
fig, ax = plt.subplots(figsize=(12, 7))

ax.plot(moneyness_range, np.array(static_lapses) * 100, 'k--', linewidth=2, label='Static (Naive)')
ax.plot(moneyness_range, np.array(linear_lapses) * 100, 'b-', linewidth=2.5, label='Linear Dynamic')
ax.plot(moneyness_range, np.array(scurve_lapses) * 100, 'r-', linewidth=2.5, label='S-Curve Dynamic')

# Reference lines
ax.axvline(x=1.0, color='gray', linestyle=':', alpha=0.7, label='ATM (Moneyness = 1.0)')
ax.axhline(y=8, color='gray', linestyle=':', alpha=0.5)

# Shade ITM vs OTM regions
ax.fill_between([0.5, 1.0], 0, 30, alpha=0.1, color='red', label='OTM (underwater)')
ax.fill_between([1.0, 1.5], 0, 30, alpha=0.1, color='green', label='ITM (in-the-money)')

ax.set_xlabel('Moneyness (Account Value / Benefit Base)', fontsize=12)
ax.set_ylabel('Annual Lapse Rate (%)', fontsize=12)
ax.set_title('Dynamic Lapse Rate Models: How Moneyness Affects Policyholder Behavior', fontsize=14)
ax.set_xlim(0.5, 1.5)
ax.set_ylim(0, 30)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Interpretation

**Key Observations:**

1. **Static model** (naive): Assumes 8% lapse regardless of conditions — unrealistic

2. **Linear model**: Simple relationship, but doesn't capture saturation effects

3. **S-curve model** (realistic):
   - Very low lapse (2-3%) when deep ITM (moneyness > 1.2)
   - Rapid increase as moneyness drops below 0.9
   - Saturates at ~25% for deep OTM (moneyness < 0.7)

## 5. Real-World Calibration

Industry studies suggest these typical parameters:

In [ ]:
# Industry-calibrated parameters
INDUSTRY_PARAMS = {
    'deep_itm_lapse': 0.02,      # 2% when deep in-the-money
    'atm_lapse': 0.06,           # 6% at-the-money (baseline)
    'deep_otm_lapse': 0.20,      # 20% when deep out-of-the-money
    'sensitivity_multiplier': 1.5, # SOA study suggests 1.5x for VA+GLWB
}

# Create industry-calibrated model
def industry_lapse_model(moneyness: float) -> float:
    """
    Industry-calibrated dynamic lapse model for VA+GLWB.
    
    Based on SOA studies and industry practice.
    """
    return scurve_dynamic_lapse(
        moneyness,
        base_rate=INDUSTRY_PARAMS['atm_lapse'],
        max_rate=INDUSTRY_PARAMS['deep_otm_lapse'],
        min_rate=INDUSTRY_PARAMS['deep_itm_lapse'],
        steepness=10.0,
        inflection=0.90,
    )

# Display calibration table
print("Industry-Calibrated Lapse Rates:")
print("-" * 40)
test_points = [0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
for m in test_points:
    lapse = industry_lapse_model(m)
    status = "OTM" if m < 1.0 else ("ATM" if m == 1.0 else "ITM")
    print(f"  {m:.1f} ({status:>3}): {lapse:>5.1%}")

## 6. Reserve Impact Analysis

How much does dynamic lapse modeling affect reserves?

In [ ]:
def simulate_reserve_impact(
    lapse_model: Callable,
    n_years: int = 20,
    n_scenarios: int = 500,
    account_value: float = 350_000,
    benefit_base: float = 350_000,
    withdrawal_rate: float = 0.05,
    equity_drift: float = 0.07,
    equity_vol: float = 0.18,
    discount_rate: float = 0.04,
) -> dict:
    """
    Simulate reserve under different lapse models.
    
    Returns statistics about reserve distribution.
    """
    dt = 1.0  # Annual steps for simplicity
    
    # Generate equity returns
    equity_returns = np.random.normal(
        equity_drift - 0.5 * equity_vol**2,
        equity_vol,
        (n_scenarios, n_years)
    )
    equity_paths = np.exp(np.cumsum(equity_returns, axis=1))
    equity_paths = np.column_stack([np.ones(n_scenarios), equity_paths])
    
    # Track policies and costs
    pv_costs = np.zeros(n_scenarios)
    in_force = np.ones(n_scenarios)  # 1 = still in force, 0 = lapsed
    av = np.full(n_scenarios, account_value)
    
    annual_withdrawal = benefit_base * withdrawal_rate
    
    for t in range(1, n_years + 1):
        # Update account values
        av = av * equity_paths[:, t] / equity_paths[:, t-1]
        av = av * 0.975  # Fees
        
        # Calculate moneyness
        moneyness = av / benefit_base
        
        # Apply dynamic lapse
        lapse_probs = np.array([lapse_model(m) for m in moneyness])
        lapses = np.random.random(n_scenarios) < lapse_probs
        
        # Process withdrawals (only for in-force policies)
        withdrawal = np.minimum(av, annual_withdrawal) * in_force
        shortfall = np.maximum(0, annual_withdrawal - av) * in_force
        av = np.maximum(0, av - annual_withdrawal)
        
        # Discount shortfall
        discount_factor = np.exp(-discount_rate * t)
        pv_costs += shortfall * discount_factor
        
        # Apply lapses
        in_force = in_force * (~lapses)
    
    # Calculate CTE70
    sorted_costs = np.sort(pv_costs)
    cte70 = sorted_costs[int(0.7 * n_scenarios):].mean()
    
    return {
        'mean': pv_costs.mean(),
        'cte70': cte70,
        'max': pv_costs.max(),
        'pct_positive': (pv_costs > 0).mean(),
        'final_in_force': in_force.mean(),
    }

# Compare models
print("Simulating reserve impact (this may take a moment)...")
print()

models = {
    'Static (8%)': lambda m: 0.08,
    'Linear Dynamic': linear_dynamic_lapse,
    'S-Curve Dynamic': scurve_dynamic_lapse,
    'Industry Calibrated': industry_lapse_model,
}

results = {}
for name, model in models.items():
    results[name] = simulate_reserve_impact(model)
    print(f"{name}: CTE70 = ${results[name]['cte70']:,.0f}")

print("\nDone!")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

model_names = list(results.keys())
cte70_values = [results[m]['cte70'] / 1000 for m in model_names]
in_force_rates = [results[m]['final_in_force'] * 100 for m in model_names]

# CTE70 comparison
colors = ['gray', 'blue', 'red', 'green']
bars1 = axes[0].bar(model_names, cte70_values, color=colors, alpha=0.8)
axes[0].set_ylabel('CTE70 Reserve ($000s)')
axes[0].set_title('Reserve Impact by Lapse Model')
axes[0].tick_params(axis='x', rotation=15)

# Add value labels
for bar, val in zip(bars1, cte70_values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                 f'${val:.0f}K', ha='center', va='bottom', fontsize=10)

# In-force comparison
bars2 = axes[1].bar(model_names, in_force_rates, color=colors, alpha=0.8)
axes[1].set_ylabel('Policies Still In-Force (%)')
axes[1].set_title('Persistency Impact by Lapse Model')
axes[1].tick_params(axis='x', rotation=15)

# Add value labels
for bar, val in zip(bars2, in_force_rates):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                 f'{val:.0f}%', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Summary table
summary_df = pd.DataFrame(results).T
summary_df['mean'] = summary_df['mean'].map(lambda x: f"${x:,.0f}")
summary_df['cte70'] = summary_df['cte70'].map(lambda x: f"${x:,.0f}")
summary_df['max'] = summary_df['max'].map(lambda x: f"${x:,.0f}")
summary_df['pct_positive'] = summary_df['pct_positive'].map(lambda x: f"{x:.1%}")
summary_df['final_in_force'] = summary_df['final_in_force'].map(lambda x: f"{x:.1%}")

summary_df.columns = ['Mean Cost', 'CTE70 Reserve', 'Max Cost', 'Pct w/ Cost', 'Final In-Force']

print("\n" + "=" * 80)
print("Reserve Impact Summary by Lapse Model")
print("=" * 80)
print(summary_df.to_string())

## 7. Sensitivity to Moneyness Scenario

Let's see how reserves change across different starting moneyness levels.

In [ ]:
# Test different starting moneyness scenarios
starting_moneyness = [0.7, 0.85, 1.0, 1.15, 1.3]
benefit_base = 350_000

moneyness_results = {'Static': [], 'Dynamic': []}

for m in starting_moneyness:
    av = benefit_base * m
    
    static_res = simulate_reserve_impact(
        lambda x: 0.08, account_value=av, benefit_base=benefit_base
    )
    dynamic_res = simulate_reserve_impact(
        industry_lapse_model, account_value=av, benefit_base=benefit_base
    )
    
    moneyness_results['Static'].append(static_res['cte70'])
    moneyness_results['Dynamic'].append(dynamic_res['cte70'])

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(starting_moneyness))
width = 0.35

bars1 = ax.bar(x - width/2, np.array(moneyness_results['Static'])/1000, width, 
               label='Static Lapse', color='gray', alpha=0.8)
bars2 = ax.bar(x + width/2, np.array(moneyness_results['Dynamic'])/1000, width,
               label='Dynamic Lapse', color='steelblue', alpha=0.8)

ax.set_xlabel('Starting Moneyness')
ax.set_ylabel('CTE70 Reserve ($000s)')
ax.set_title('Reserve by Starting Moneyness: Static vs Dynamic Lapse')
ax.set_xticks(x)
ax.set_xticklabels([f'{m:.2f}' for m in starting_moneyness])
ax.legend()

# Add difference annotations
for i, (s, d) in enumerate(zip(moneyness_results['Static'], moneyness_results['Dynamic'])):
    diff_pct = (d - s) / s * 100
    ax.annotate(f'{diff_pct:+.0f}%', xy=(i, max(s, d)/1000 + 2),
                ha='center', fontsize=9, color='red' if diff_pct > 0 else 'green')

plt.tight_layout()
plt.show()

## 8. Key Takeaways

### Behavior Modeling Matters

1. **Static lapse is wrong**: Overstates reserves for ITM, understates for OTM

2. **Dynamic lapse captures reality**: Policyholders respond rationally to market conditions

3. **Reserve impact is material**: 10-30% difference depending on starting moneyness

4. **Industry calibration essential**: Parameters should reflect observed behavior data

### Practical Implications

| Scenario | Static Lapse | Dynamic Lapse | Business Impact |
|----------|--------------|---------------|------------------|
| ITM Portfolio | Over-reserved | Accurate | Capital efficiency |
| OTM Portfolio | Under-reserved | Accurate | Risk management |
| Mixed Portfolio | Averaged out | Precise by segment | Better pricing |

### InsuranceAI Toolkit Application

The **InsuranceAI Toolkit** implements these models:
- Automatic moneyness calculation
- S-curve dynamic lapse with industry calibration
- Reserve sensitivity analysis
- Integration with hedging workflows

Try it: [InsuranceAI Toolkit Demo](https://insurance-ai-toolkit.streamlit.app)

---

## References

1. SOA: Policyholder Behavior in the Tail Study (2012)
2. AAA: VM-21 Practice Note on Policyholder Behavior
3. Kling, Ruez, Russ (2011): "The Impact of Policyholder Behavior on Pricing, Hedging, and Hedge Efficiency of Withdrawal Benefit Guarantees in Variable Annuities"

---

*Created by Brandon Behring | [LinkedIn](https://www.linkedin.com/in/brandon-behring/) | [InsuranceAI Toolkit](https://github.com/brandonmbehring-dev/insurance-ai-toolkit)*